In [3]:
import motmetrics as mm
import numpy as np
import json


In [29]:
def get_summary_dataset_mot(gt_path,dataset,detector,tracker):
    gt={}
    json_ground_trth = "../%s.txt"%gt_path
    gt_file = open(json_ground_trth,"r")
    line = gt_file.readline()
    while(line):
        line = line.replace("\n", "")
        comps = line.split(",")
        image_id = int(comps[0])
        if(image_id not in gt):
            gt[image_id]= []
        gt[image_id].append(np.array([int(comps[1]),float(comps[2]),float(comps[3]),float(comps[4]),float(comps[5])],np.float32))
        line = gt_file.readline()
    gt_file.close()
            
    json_results = "%s/%s/%s.json"%(dataset,detector,tracker)
    dets = {}
    with open(json_results) as json_file:  
        data = json.load(json_file)
        for box in data:
            image_id=box['image_id']
            bbox = box['bbox']
            if(image_id in dets):
                l = list(map(float, bbox))
                l.append(box['id'])
                dets[image_id].append(l)
                
            else:
                dets[image_id]=[]
                l = list(map(float, bbox))
                l.append(box['id'])
                dets[image_id].append(l)
                
    
    acc = mm.MOTAccumulator(auto_id=True)
    
    for key, value in gt.items():
        if(key in dets):
            track_boxes = [v[0:4] for v in dets[key]]
            ids = [d[4] for d in dets[key]]
        else:
            track_boxes = []
            ids=[]
        
        gt_boxes = [vv[1:] for vv in value]
        
        ff = mm.distances.iou_matrix(track_boxes,gt_boxes, max_iou=0.5)
        
            
        frameId =acc.update(
        [t[0] for t in value] ,                 # Ground truth objects in this frame
        ids,                  # Detector hypotheses in this frame

                 ff)
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=['num_frames', 'num_false_positives','num_misses','num_switches','num_matches','mostly_tracked','partially_tracked','mostly_lost','precision','recall','mota', 'motp','obj_frequencies','pred_frequencies','num_detections','num_objects'], name='acc')
    return summary

In [32]:
def get_summary_dataset(gt_path,dataset,detector,tracker):
    gt={}
    json_ground_trth = "../%s.json"%gt_path
    with open(json_ground_trth) as json_file:  
        data = json.load(json_file)
        for box in data['annotations']:
            image_id=box['image_id']
            bbox = box['bbox']
            if(image_id in gt):
                gt[box['image_id']].append(list(map(float, bbox)))
            else:
                gt[box['image_id']]=[]
                gt[box['image_id']].append(list(map(float, bbox)))
    json_results = "%s/%s/%s.json"%(dataset,detector,tracker)
    dets = {}
    with open(json_results) as json_file:  
        data = json.load(json_file)
        for box in data:
            image_id=box['image_id']
            bbox = box['bbox']
            if(image_id in dets):
                l = list(map(float, bbox))
                l.append(box['id'])
                dets[image_id].append(l)
                
            else:
                dets[image_id]=[]
                l = list(map(float, bbox))
                l.append(box['id'])
                dets[image_id].append(l)
    acc = mm.MOTAccumulator(auto_id=True)
    print(dets)
    for key, value in gt.items():
        
        if(key in dets):
            track_boxes = [v[0:4] for v in dets[key]]
            ids = [d[4] for d in dets[key]]
        else:
            track_boxes = []
            ids=[]
    
        
        ff = mm.distances.iou_matrix(track_boxes, value, max_iou=0.5)
        print(ff)
            
        frameId =acc.update(
        np.arange(len(value)),                 # Ground truth objects in this frame
        ids),                  # Detector hypotheses in this frame

                 ff)
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=['num_frames', 'num_false_positives','num_misses','num_switches','num_matches','mostly_tracked','partially_tracked','mostly_lost','precision','recall','mota', 'motp','obj_frequencies','pred_frequencies','num_detections','num_objects'], name='acc')
    return summary

IndentationError: unexpected indent (<ipython-input-32-fbb5f482a4cc>, line 50)

In [36]:
import os
import csv
ground_truth_file_names = ['data_graal2']
dataset_output_path = ['graal_2_videos']
for i in range(len(ground_truth_file_names)):
    
    with open('%s/mot_results.csv'%(dataset_output_path[i]), 'w', newline='') as csvFile:
        writer = csv.writer(csvFile)
        row = ['Frames','Tot Objects','Tot detections','TP','FN','FP','Track Switches','Precision','Recall','MOTA','MOTP']
        writer.writerow(row)
        for det in ['yolo','retinanet']:
            for track in ['baseline','center_flow','keypoint_flow','kalman_center','kalman_corners','SORT']:
                summary = get_summary_dataset(ground_truth_file_names[i],dataset_output_path[i],det,track)
                
                row = []
                row.append(summary['num_frames'][0])
                row.append(summary['num_objects'][0])
                row.append(summary['num_detections'][0])
                row.append(summary['num_matches'][0])
                row.append(summary['num_misses'][0])
                row.append(summary['num_false_positives'][0])
                row.append(summary['num_switches'][0])
                row.append(summary['precision'][0])
                row.append(summary['recall'][0])
                row.append(summary['mota'][0])
                row.append(summary['motp'][0])
                writer.writerow(row)



    csvFile.close()

{1: [[140.0, 328.0, 277.0, 81.0], [659.0, 321.0, 372.0, 116.0], [394.0, 622.0, 214.0, 151.0]], 2: [[143.0, 327.0, 271.0, 82.0], [663.0, 275.0, 363.0, 149.0], [398.0, 624.0, 208.0, 148.0]], 3: [[140.0, 310.0, 279.0, 90.0], [660.0, 276.0, 371.0, 145.0], [393.0, 624.0, 217.0, 148.0], [0.0, 353.0, 61.0, 25.0]], 4: [[142.0, 309.0, 276.0, 89.0], [393.0, 624.0, 218.0, 147.0], [659.0, 279.0, 365.0, 140.0], [0.0, 351.0, 63.0, 26.0]], 5: [[148.0, 305.0, 271.0, 94.0], [392.0, 624.0, 219.0, 146.0], [671.0, 279.0, 354.0, 140.0]], 6: [[150.0, 308.0, 271.0, 88.0], [665.0, 280.0, 362.0, 138.0], [394.0, 628.0, 216.0, 143.0], [0.0, 344.0, 68.0, 28.0]], 7: [[151.0, 308.0, 272.0, 88.0], [662.0, 284.0, 369.0, 129.0], [399.0, 628.0, 207.0, 144.0], [0.0, 344.0, 69.0, 29.0]], 8: [[157.0, 304.0, 261.0, 94.0], [671.0, 281.0, 356.0, 131.0], [399.0, 625.0, 209.0, 147.0], [0.0, 351.0, 70.0, 25.0]], 9: [[157.0, 305.0, 260.0, 93.0], [392.0, 625.0, 217.0, 146.0], [664.0, 285.0, 367.0, 129.0], [0.0, 353.0, 72.0, 24.0]

 [       nan        nan]]
[[       nan 0.42093793]
 [       nan        nan]]
[[       nan 0.40841538]
 [       nan        nan]]
[[       nan 0.38323171]
 [0.44733514        nan]
 [       nan        nan]]
[[       nan 0.31793342]]
[[       nan 0.35210846]
 [       nan        nan]]
[[       nan 0.33600049]
 [       nan        nan]
 [0.4233497         nan]]
[[       nan 0.32014657]
 [       nan        nan]]
[[       nan 0.33181049]
 [       nan        nan]]
[[       nan 0.31018055]
 [       nan        nan]
 [       nan        nan]]
[[       nan 0.34114921]
 [       nan        nan]
 [       nan        nan]]
[[       nan 0.37271751]
 [       nan        nan]]
[[       nan 0.34070718]
 [       nan        nan]]
[[       nan 0.31383432]
 [       nan        nan]]
[[       nan 0.32398118]
 [       nan        nan]]
[[       nan 0.32135216]
 [       nan        nan]
 [       nan        nan]]
[[       nan 0.29010908]
 [       nan        nan]]
[[       nan 0.28945901]
 [       nan        nan]]
[[     

 [       nan        nan]]
[[       nan 0.16890145]
 [       nan        nan]]
[[       nan 0.11471165]
 [       nan        nan]]
[[      nan 0.1543099]
 [      nan       nan]]
[[       nan 0.21568804]
 [       nan        nan]]
[[       nan 0.20306655]
 [       nan        nan]]
[[       nan 0.16164686]
 [       nan        nan]]
[[     nan      nan]
 [     nan 0.133072]]
[[       nan 0.10808281]
 [       nan        nan]]
[[       nan 0.14063328]
 [       nan        nan]]
[[       nan 0.12895442]
 [       nan        nan]]
[[       nan 0.11298197]
 [       nan        nan]]
[[       nan 0.10788417]
 [       nan        nan]]
[[       nan 0.08877759]
 [       nan        nan]
 [       nan        nan]]
[[       nan 0.14025303]
 [       nan        nan]]
[[       nan 0.22750588]
 [       nan        nan]
 [       nan        nan]]
[[       nan 0.16317269]
 [       nan        nan]]
[[       nan 0.17072753]
 [       nan        nan]]
[[       nan 0.22911331]
 [       nan        nan]]
[[       nan 0.256

[[       nan        nan        nan]
 [       nan        nan 0.40800526]
 [       nan        nan        nan]]
[[       nan        nan        nan]
 [       nan        nan 0.39936548]
 [       nan        nan        nan]]
[[       nan        nan        nan]
 [       nan        nan 0.36261472]
 [       nan        nan        nan]]
[[       nan        nan        nan]
 [       nan        nan 0.33877392]
 [       nan        nan        nan]]
[[       nan        nan        nan]
 [       nan        nan 0.33492719]
 [       nan        nan        nan]]
[[       nan        nan        nan]
 [       nan        nan 0.32894737]
 [       nan        nan        nan]]
[[       nan        nan        nan]
 [       nan        nan 0.40121189]
 [       nan        nan        nan]]
[[       nan        nan        nan]
 [       nan        nan 0.46540084]
 [       nan        nan        nan]]
[[       nan        nan        nan]
 [       nan        nan 0.45697304]
 [       nan        nan        nan]]
[[       nan       

 [       nan        nan]]
[[       nan 0.26542218]
 [       nan        nan]
 [0.43251293        nan]]
[[       nan 0.34605405]
 [       nan        nan]
 [0.45944671        nan]]
[[       nan 0.38263942]
 [       nan        nan]
 [0.44723171        nan]]
[[       nan 0.39038417]
 [       nan        nan]
 [0.49154775        nan]]
[[       nan 0.36390533]
 [       nan        nan]
 [0.49154775        nan]]
[[       nan 0.35721058]
 [       nan        nan]
 [0.49462606        nan]]
[[       nan 0.35650224]
 [       nan        nan]
 [0.49462606        nan]]
[[       nan 0.35540442]
 [       nan        nan]
 [0.48782293        nan]]
[[       nan 0.29400134]
 [       nan        nan]
 [0.49333726        nan]]
[[       nan 0.30206805]
 [       nan        nan]
 [       nan        nan]]
[[       nan 0.31603692]
 [       nan        nan]
 [       nan        nan]]
[[       nan 0.33780731]
 [       nan        nan]
 [0.47895208        nan]]
[[       nan 0.35523614]
 [       nan        nan]
 [0.44767876

...

 [0.35963823        nan]]
[[       nan 0.31505093]
 [       nan        nan]
 [0.41167024        nan]]
[[       nan 0.245898  ]
 [       nan        nan]
 [0.37698413        nan]]
[[       nan 0.26273275]
 [       nan        nan]
 [0.39922481        nan]]
[[       nan 0.29648024]
 [       nan        nan]
 [0.46445165        nan]]
[[       nan 0.27389906]
 [       nan        nan]
 [0.48534704        nan]]
[[       nan 0.32979593]
 [       nan        nan]
 [0.49646689        nan]]
[[      nan 0.3606814]
 [      nan       nan]
 [      nan       nan]]
[[       nan 0.40853128]
 [       nan        nan]
 [       nan        nan]]
[[       nan 0.37635377]
 [       nan        nan]
 [0.40854326        nan]]
[[      nan 0.2626893]
 [      nan       nan]
 [      nan       nan]]
[[       nan 0.39187608]
 [       nan        nan]
 [       nan        nan]]
[[       nan 0.45604396]
 [       nan        nan]
 [       nan        nan]]
[[     nan 0.243031]
 [     nan      nan]
 [     nan      nan]]
[[       n

[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[]
[]
[]
[]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]
 [nan]]
[[nan]
 [nan]]
[[nan]
 [nan]]
[[nan]
 [nan]]
[[nan]
 [nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]
 [nan]]
[[nan]
 [nan]]
[[nan]
 [nan]]
[[nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]]
[[nan]
 [nan]]
[[nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]]
[[nan]
 [nan]]
[[nan]
 [nan]]
[[nan]
 [nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
{4: [[155.0, 319.0, 257.0, 88.0], [370.0, 606.0, 247.0, 169.0], [664.0, 232.0, 362.0, 207.0]], 5: [[153.0, 298.0, 262.0, 106.0], [372.0, 605.0, 244.0, 169.0], [666.0, 260.0, 360.0, 172.0]], 6: [[154.0, 307.0, 259.0, 97.0], [372.0, 604.0, 245.0, 170.0], [668.0, 280.0, 359.0, 161.0]], 7: [[156.0, 308.0, 261.0, 97.0], [374.0, 606.0, 243.0, 169.0], [677.0, 313.0, 348.0, 126.0]], 8: [[159.0, 310.0, 260.0, 95.0], [374.0, 606.0, 244.0, 170.0], [676.0, 302.0, 355.0, 138.0]], 9:

 [       nan        nan]]
[[nan nan]
 [nan nan]
 [nan nan]]
[[nan nan]
 [nan nan]
 [nan nan]]
[[       nan 0.40682847]
 [       nan        nan]
 [       nan        nan]]
[[nan nan]
 [nan nan]]
[[       nan 0.30349195]
 [       nan        nan]]
[[       nan 0.45982906]
 [       nan        nan]]
[[       nan 0.41701975]
 [       nan        nan]]
[[       nan 0.30267703]
 [       nan        nan]]
[[       nan 0.34594499]
 [       nan        nan]]
[[       nan 0.36773519]
 [       nan        nan]]
[[       nan 0.37792319]
 [       nan        nan]]
[[       nan 0.42082741]
 [       nan        nan]]
[[       nan 0.37602105]
 [       nan        nan]]
[[       nan 0.36339406]
 [       nan        nan]]
[[     nan 0.354078]
 [     nan      nan]]
[[       nan 0.32829555]
 [       nan        nan]]
[[       nan 0.39614376]
 [       nan        nan]]
[[       nan 0.49309479]
 [       nan        nan]]
[[       nan 0.44691501]
 [       nan        nan]]
[[nan nan]
 [nan nan]]
[[       nan 0.47428571]
 [

[[     nan 0.190796]
 [     nan      nan]]
[[       nan 0.13022125]
 [       nan        nan]]
[[       nan 0.19733439]
 [       nan        nan]]
[[       nan 0.20625843]
 [       nan        nan]]
[[       nan 0.15170361]
 [       nan        nan]]
[[       nan 0.22932836]
 [       nan        nan]]
[[       nan 0.22927669]
 [       nan        nan]]
[[       nan 0.16526069]
 [       nan        nan]]
[[       nan 0.16522614]
 [       nan        nan]]
[[       nan 0.17297372]
 [       nan        nan]]
[[       nan 0.24955786]
 [       nan        nan]
 [0.42626514        nan]]
[[       nan 0.30201574]
 [       nan        nan]
 [0.45510708        nan]]
[[       nan 0.29539703]
 [       nan        nan]
 [0.48973904        nan]]
[[       nan 0.32475873]
 [0.46227533        nan]]
[[       nan 0.17661362]
 [       nan        nan]]
[[       nan 0.18206025]]
[[      nan 0.1450701]]
[[       nan 0.18224405]]
[[       nan 0.16930393]]
[[       nan 0.16154066]]
[[       nan 0.16687501]]
[[       nan 0

 [       nan 0.30662824]]
[[     nan      nan]
 [     nan 0.191124]]
[[nan nan]]
[[nan nan]]
[[nan nan]]
[[      nan 0.2446814]]
[[       nan 0.28170203]]
[[      nan 0.2057554]]
[[       nan 0.20647428]]
[[       nan 0.19919633]]
[[0.37454279        nan]
 [       nan 0.20255864]]
[[       nan 0.26404423]]
[[       nan 0.30798479]]
[[       nan 0.32953273]]
[[       nan 0.33905405]]
[[       nan 0.29259259]]
[[       nan 0.20276895]]
[[       nan        nan]
 [0.35380255        nan]
 [       nan 0.20647121]]
[[       nan        nan]
 [       nan 0.26279474]]
[[       nan        nan]
 [       nan 0.20536669]]
[[       nan        nan]
 [       nan 0.24238939]]
[[       nan        nan]
 [       nan 0.22333496]]
[[       nan        nan]
 [       nan 0.21699053]]
[[       nan 0.23888189]]
[[       nan 0.25900117]]
[[       nan 0.21263197]]
[[       nan        nan]
 [       nan 0.23543099]]
[[       nan 0.21147488]]
[[       nan 0.25771239]]
[[       nan 0.29074865]]
[]
[]
[]
[]
[]
[]
[]
[]


In [30]:
summary = get_summary_dataset_mot('mot_gt','mot_1_videos','def','keypoint_flow')

In [31]:
row = ['Frames','Tot Objects','Tot detections','TP','FN','FP','Track Switches','Precision','Recall','MOTA','MOTP']
print(row)
row = []
row.append(summary['num_frames'][0])
row.append(summary['num_objects'][0])
row.append(summary['num_detections'][0])
row.append(summary['num_matches'][0])
row.append(summary['num_misses'][0])
row.append(summary['num_false_positives'][0])
row.append(summary['num_switches'][0])
row.append(summary['precision'][0])
row.append(summary['recall'][0])
row.append(summary['mota'][0])
row.append(summary['motp'][0])
print(row)

['Frames', 'Tot Objects', 'Tot detections', 'TP', 'FN', 'FP', 'Track Switches', 'Precision', 'Recall', 'MOTA', 'MOTP']
[795, 4650, 2211, 1887, 2439, 1411, 324, 0.6104362230811706, 0.4754838709677419, 0.10236559139784951, 0.33281533317053325]
